In [1]:
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.datasets import imdb

Using Theano backend.


In [2]:
max_features = 20000
maxlen = 80  # cut texts after this number of words (among top max_features most common words)
batch_size = 32

In [3]:
print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(nb_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

Loading data...
25000 train sequences
25000 test sequences


In [23]:
set(y_test)

{0, 1}

In [4]:
from itertools import chain
max_features = len(set(chain.from_iterable(X_train)) and set(chain.from_iterable(X_test)))

In [27]:
from keras.layers import Convolution1D , GlobalMaxPooling1D
from keras.layers.convolutional import Conv1D
num_filters = 150
kernel_size = 3
filter_length = 3

むしろ
http://qiita.com/HirofumiYashima/items/2aee883b736ee600899b
あるいは
http://qiita.com/HirofumiYashima/items/a01902cb7d3e74b56bcb
を参考にCNNを構成しよう。後者がstraightforwardか？？

In [28]:
model = Sequential()
model.add(Embedding(max_features, 128, dropout=0.2))

filter : アウトプット次元（＝フィルターの数）
kernel_size :ウィンドウのサイズ

In [29]:
model.add(Convolution1D(nb_filter = num_filters , filter_length = filter_length))
#model.add(Conv1D(filter = num_filters , kernel_size = kernel_size))

In [30]:
# we use max pooling: 
model.add(GlobalMaxPooling1D()) 

hidden_dims = 250 
# We add a vanilla hidden layer: 
model.add(Dense(hidden_dims)) 
model.add(Dropout(0.2)) 
model.add(Activation('relu')) 
# We project onto a single unit output layer, and squash it with a sigmoid: 
model.add(Dense(1)) 
model.add(Activation('sigmoid')) 

In [31]:
if False:
    model.add(LSTM(128, dropout_W=0.2, dropout_U=0.2))  # try using a GRU instead, for fun

In [32]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
embedding_6 (Embedding)          (None, None, 128)     2494720     embedding_input_6[0][0]          
____________________________________________________________________________________________________
convolution1d_5 (Convolution1D)  (None, None, 150)     57750       embedding_6[0][0]                
____________________________________________________________________________________________________
globalmaxpooling1d_1 (GlobalMaxPo(None, 150)           0           convolution1d_5[0][0]            
____________________________________________________________________________________________________
dense_3 (Dense)                  (None, 250)           37750       globalmaxpooling1d_1[0][0]       
___________________________________________________________________________________________

In [33]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy']) 

In [ ]:
nb_epoch = 2
model.fit(X_train, y_train, 
          batch_size=batch_size, 
          nb_epoch=nb_epoch, 
          validation_data=(X_test, y_test) , verbose = 2) 